# imports and settings

In [73]:
import numpy as np
import anndata as ad
import scanpy as sc
import palettable
import time
import seaborn as sns

In [74]:
%matplotlib inline

# sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3
heatmap_cmp = palettable.cmocean.diverging.Balance_20.mpl_colormap
# sc.set_figure_params(fontsize=20)


sns.set_style('white')
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 2}

# matplotlib.rc('font', **font)
# matplotlib.rcParams.update({'font.size': 2})

import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title







# function

In [75]:
from sklearn.preprocessing import *
import scanpy as sc
import numpy as np
import time
from skimage.color import rgb2lab, lab2rgb
from scipy.sparse import issparse 
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import anndata as ad
import matplotlib.pyplot as plt

def SOViewer_UMAP(adata,pca=100,dot_size=0.001,marker='o',plot_cluster=None):
    # 输入：
    # adata(附带spatial, 处理过后的X)
    # sample_rate：默认0.01


    print('projecting all data into PCA space...')
    time_start=time.time()
    
    sc.pp.pca(adata,n_comps=pca)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata,n_components=3)
    test_embedding = adata.obsm['X_umap']
    time_end=time.time()
    print('projecting time cost',time_end-time_start,'s')

    print('generating color coding...')
    rgb = MinMaxScaler(clip=True).fit_transform(test_embedding)
    lab = np.zeros_like(rgb)
    lab[:,0] = MinMaxScaler(feature_range=(0, 100),clip=True).fit_transform(rgb[:,0][:,None])[:,0]
    lab[:,1] = MinMaxScaler(feature_range=(-128, 127),clip=True).fit_transform(rgb[:,1][:,None])[:,0]
    lab[:,2] = MinMaxScaler(feature_range=(-128, 127),clip=True).fit_transform(rgb[:,2][:,None])[:,0]
    lab = lab2rgb(lab)

    c = lab
    print(c.max(),c.min())
    plt.scatter(x=adata.obsm['spatial'][:,0],y=adata.obsm['spatial'][:,1],c=c,s=dot_size,edgecolors='none',marker=marker)
    plt.gca().set_aspect('equal', adjustable='box')
#     plt.title('lab')
    plt.title('NO sampling, Umap, LAB')

    plt.show()

    c = rgb
    print(c.max(),c.min())
    
    plt.scatter(x=adata.obsm['spatial'][:,0],y=adata.obsm['spatial'][:,1],c=c,s=dot_size,edgecolors='none',marker=marker)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('NO sampling, Umap, RGB')
    plt.show()
    
    if plot_cluster is None:
        return
    c = np.array(adata.obs[plot_cluster]).astype('int')
    
    plt.scatter(x=adata.obsm['spatial'][:,0],y=adata.obsm['spatial'][:,1],c=c,s=dot_size,edgecolors='none',marker=marker)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('cluster on spatial')
    plt.show()
        


# data loading

In [76]:
#     data_name = '07052018_032018_88B_5'
import os
def save_tMaldi_imaging_data(
    data_name,
    base_path='/home/yzy/PUBDT/st/IMS/tMALDI',  
):
    print('processing data: {0}'.format(data_name))
    # specify data paths
    example_path = base_path
    data_file = '{0}/data/{1}.imzML'.format(example_path,data_name)

    output_image_root = example_path+'/output_image'
    output_image_folder = '{0}/{1}'.format(output_image_root,data_name)

    output_view_folder = example_path+'/output_view'

    output_h5ad_folder = example_path+'/h5ad'
    isExist = os.path.exists(output_image_folder)

    if not isExist:
        os.makedirs(output_image_folder)
      # Create a new directory because it does not exist 
        print("{0} does not exist, created one.".format(output_image_folder))
    else:
        print('{0} existed.'.format(output_image_folder))


    # generate adata
    from pyimzml.ImzMLParser import ImzMLParser

    p = ImzMLParser(data_file)

    mzs_list = []
    spatial_list = []
    for idx, (x,y,z) in enumerate(p.coordinates):

        mzs, intensities = p.getspectrum(idx)
        mzs_list.extend(mzs)
        spatial_list.append([x,y])
    spatial_list = np.array(spatial_list)

    mzs_int_list = np.unique(np.floor((mzs_list)))
    mzs_int_reverse_dict = dict(zip(mzs_int_list,np.arange(len(mzs_int_list))))
    X = np.zeros(shape=(spatial_list.shape[0],len(mzs_int_list)))
    for idx, (x,y,z) in enumerate(p.coordinates):

        mzs, intensities = p.getspectrum(idx)
        for i in range(len(mzs)):
            cur_mz = mzs[i]
            cur_intensity = intensities[i]
            cur_mz_int = np.floor(cur_mz)
            X[idx,mzs_int_reverse_dict[cur_mz_int]] += cur_intensity
    adata = ad.AnnData(X)
    adata.var_names = mzs_int_list.astype('str')
    adata.obsm['spatial'] = spatial_list
    adata.write_h5ad('{0}/{1}.h5ad'.format(output_h5ad_folder,data_name))
    # save imaging data
#     x_array = adata.obsm['spatial'][:,0]
#     y_array = adata.obsm['spatial'][:,1]
#     dot_size = 80
#     marker = 's'
#     save = output_image_folder
#     for plot_var in adata.var_names:
#         c = np.array(adata[:,plot_var].X).reshape(-1,)

#         plt.scatter(x=x_array,y=y_array,c=c,s=dot_size,edgecolors='none',marker=marker)
#         plt.gca().set_aspect('equal', adjustable='box')
#         plt.title('{0} : {1} m/z'.format(data_name,plot_var))
#         if save is None:
#             plt.show()
#         else:
#             plt.savefig('{0}/{1}.png'.format(save,plot_var),format='png')


            
#     save = output_image_folder
#     x_array = adata.obsm['spatial'][:,0]
#     y_array = adata.obsm['spatial'][:,1]
#     shape_x = x_array.max()
#     shape_y = y_array.max()
#     for plot_var in adata.var_names:
#         img = np.array(adata[:,plot_var].X).reshape(shape_y,shape_x)
#         plt.imshow(img)
#         plt.axis('off')
#         plt.savefig('{0}/{1}.png'.format(save,plot_var),format='png')


In [69]:
example_path = '/home/yzy/PUBDT/st/IMS/tMALDI'

In [77]:
for file in os.listdir(example_path+'/data'):
    if not file.endswith('.ibd'):
        continue
    data_name = file[:-4]
    save_tMaldi_imaging_data(data_name,example_path)
    

processing data: Fig2_1um_cerebellum_pos
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig2_1um_cerebellum_pos does not exist, created one.
processing data: Fig4ii_0d8um_cerebellum_pos_tMALDI2
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig4ii_0d8um_cerebellum_pos_tMALDI2 does not exist, created one.
processing data: Fig3iii_1um_cerebellum_pos
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig3iii_1um_cerebellum_pos does not exist, created one.
processing data: Fig4v_2um_cerebellum_pos_tMALDI2
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig4v_2um_cerebellum_pos_tMALDI2 does not exist, created one.
processing data: FigS2_1um_MALDI_MALDI2
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/FigS2_1um_MALDI_MALDI2 does not exist, created one.
processing data: Fig4iv_1d5um_cerebellum_pos_tMALDI2
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig4iv_1d5um_cerebellum_pos_tMALDI2 does not exist, created one.
processing data: Fig4i_0d6um_cerebellum_pos_tMALDI2
/home/yzy/PUBDT/st/IMS/tMALDI/output_image/Fig4i_0d6um_c